To begin with we need to import the models we are testing, and some additional libraries/functions

In [1]:
from models import ClassifierWrapper, ClusteringWrapper, train_model,set_defaults
from datagen import generate_classification_data, generate_clustering_data
from sklearn.model_selection import train_test_split



Now we define our syntethic dataset

In [12]:
classifierXY = generate_classification_data(n_samples=10_000, n_features=20, n_informative=15, n_classes=2,sparsity=0.0, random_state=42)
classifierSplit = train_test_split(classifierXY[0], classifierXY[1], test_size=0.2, random_state=42)
clusteringXY = generate_clustering_data(n_samples=100_000, n_features=50, centers=10, cluster_std=1.0, random_state=42)


Subsequently we define the model parameters and models used for testing.


In [13]:
SEED = 42

setting={
    "classifier": {
        "estimator_name": "random_forest",
        "use_scaler": True,
        "random_state": SEED,
        "n_estimators": 100,
        "max_iter": 1000,
        "probability": True,
    },
    "clustering": {
        "algorithm_name": "kmeans",
        "n_clusters": 3,
        "use_scaler": True,
        "random_state": SEED,
        "algorithm_params": None,
    },
}
set_defaults(setting)

rfc = ClassifierWrapper()
kmc = ClusteringWrapper()

In [15]:
resultClassifier=train_model(model=rfc,X=classifierSplit[0],y=classifierSplit[2],X_val=classifierSplit[1],y_val=classifierSplit[3])
resultClustering=train_model(kmc,X=clusteringXY[0])
print("Classifier results:", resultClassifier)
print("Clustering results:", resultClustering)

Classifier results: {'times': [2.9405687610014866], 'median': 2.9405687610014866, 'mean': 2.9405687610014866, 'std': 0.0}
Clustering results: {'times': [0.2930060729995603], 'median': 0.2930060729995603, 'mean': 0.2930060729995603, 'std': 0.0}
